### Importing Libraries
This block imports the necessary libraries and modules for handling datasets, tokenization, and model training using Hugging Face's Transformers library.

In [ ]:
import pandas as pd  # For data manipulation
import torch  # For tensor operations
from datasets import Dataset  # For managing datasets in Hugging Face format
from transformers import (  # Hugging Face library for NLP
    BartTokenizerFast,  # Tokenizer for BART
    BartForConditionalGeneration,  # Pretrained BART model for sequence generation
    TrainingArguments,  # Configuration for training
    Trainer,  # Training wrapper
    DataCollatorForSeq2Seq,  # Prepares data batches for sequence-to-sequence tasks
)


/home/toru/anaconda3/envs/polarity_env_pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading and Preprocessing
This block loads the dataset, verifies its integrity, ensures proper data types, and prepares a new column with task-specific prefixes for input.

In [ ]:
# Load the dataset
data = pd.read_csv('finaldataset.csv')  # Read dataset from CSV file

# Display the first few rows to verify the data
print(data.head())  # Check the structure of the dataset

# Check for any missing or malformed entries
print("Checking for missing values:")
print(data.isnull().sum())  # Count missing values in each column

# Ensure all reviews are strings
data['original_review'] = data['original_review'].astype(str)  # Convert to string type
data['inverted_review'] = data['inverted_review'].astype(str)  # Convert to string type

# Add a task-specific prefix to the original reviews
data['input_text'] = 'invert polarity: ' + data['original_review']  # Add task-specific prompt

# Convert the DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data[['input_text', 'inverted_review']])  # Keep only relevant columns


                                     original_review  original_label  \
0  Great CD After buying this CD for its hype, I ...               1   
1       Blends in seamlessly with my car’s interior.               1   
2  Buy the soundtrack instead. Joseph lovers; you...               0   
3  So cute! This hat is honestly one of the very ...               1   
4  wish i had gotten this sooner! ive got a lamp ...               1   

                                     inverted_review  inverted_label  
0  Terrible CD After buying this CD due to its hy...               0  
1    Stands out awkwardly against my car’s interior.               0  
2  This film is a delightful experience that brin...               1  
3  So ugly! This hat is honestly one of the very ...               0  
4  I regret getting this at all! The lamp in the ...               0  
Checking for missing values:
original_review    0
original_label     0
inverted_review    0
inverted_label     0
dtype: int64


### Splitting the Dataset
This block splits the dataset into training and validation sets for model training and evaluation.

In [ ]:
# Split the dataset into training and validation sets
tokenized_dataset = hf_dataset.train_test_split(test_size=0.1)  # Allocate 10% of the data for validation
train_dataset = tokenized_dataset['train']  # Training subset
eval_dataset = tokenized_dataset['test']  # Validation subset

### Loading Pretrained Model and Tokenizer
This block loads the pretrained BART model and its tokenizer for sequence-to-sequence tasks.

In [ ]:
# Load the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')  # Load BART tokenizer

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')  # Load BART model

### Preprocessing Function
Defines a function to tokenize input and output texts and prepare them for the model, handling padding and special tokens appropriately.

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=128)  # Tokenize input
    outputs = tokenizer(examples['inverted_review'], padding='max_length', truncation=True, max_length=128)  # Tokenize output

    examples['input_ids'] = inputs.input_ids  # Save input token IDs
    examples['attention_mask'] = inputs.attention_mask  # Save attention mask
    examples['labels'] = outputs.input_ids  # Save output token IDs
    # Replace padding token IDs in labels with -100 to ignore them during loss computation
    examples['labels'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels]
        for labels in examples['labels']
    ]
    return examples

### Applying Preprocessing
Maps the preprocessing function to the datasets and removes unnecessary columns.

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['input_text', 'inverted_review'])  # Preprocess training data
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=['input_text', 'inverted_review'])  # Preprocess validation data

Map: 100%|██████████| 724/724 [00:00<00:00, 10220.99 examples/s]


### Formatting Datasets
This block ensures the datasets are formatted as PyTorch tensors with only relevant columns for training.

In [ ]:
columns = ['input_ids', 'attention_mask', 'labels']  # Required columns
train_dataset.set_format(type='torch', columns=columns)  # Format training dataset
eval_dataset.set_format(type='torch', columns=columns)  # Format evaluation dataset

### Training Arguments
Defines training hyperparameters and configurations for the Hugging Face Trainer.

In [ ]:
training_args = TrainingArguments(
    output_dir='./polarity_inversion_model',  # Directory to save model outputs
    num_train_epochs=10,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save model checkpoints at the end of each epoch
    learning_rate=3e-5,  # Learning rate
    warmup_steps=100,  # Warmup steps for learning rate scheduler
    logging_dir='./logs',  # Directory for logging
    logging_steps=50,  # Log every 50 steps
    save_total_limit=2,  # Maximum number of checkpoints to save
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU supports it
    report_to="none",  # Disable reporting to external systems
)

/home/toru/anaconda3/envs/polarity_env_pytorch/lib/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Data Collator
Creates a data collator to dynamically pad batches during training and evaluation.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  # Prepares batches with dynamic padding

### Trainer Initialization
Initializes the Hugging Face Trainer for managing the training and evaluation loops.

In [ ]:
trainer = Trainer(
    model=model,  # Model to be fine-tuned
    args=training_args,  # Training configuration
    train_dataset=train_dataset,  # Training data
    eval_dataset=eval_dataset,  # Evaluation data
    data_collator=data_collator,  # Collator for batch preparation
)

### Model Training and Saving
This block trains the model and saves the fine-tuned model and tokenizer to the specified directory.

In [ ]:
# Start the fine-tuning process
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('./polarity_inversion_model')  # Save model weights
tokenizer.save_pretrained('./polarity_inversion_model')  # Save tokenizer

  0%|          | 0/16270 [00:00<?, ?it/s]/home/toru/anaconda3/envs/polarity_env_pytorch/lib/python3.9/site-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
  0%|          | 51/16270 [00:04<25:56, 10.42it/s]

{'loss': 2.7809, 'grad_norm': 11.615219116210938, 'learning_rate': 1.4099999999999999e-05, 'epoch': 0.03}


  1%|          | 101/16270 [00:09<26:08, 10.31it/s]

{'loss': 2.0501, 'grad_norm': 8.195572853088379, 'learning_rate': 2.91e-05, 'epoch': 0.06}


  1%|          | 151/16270 [00:14<25:57, 10.35it/s]

...

                                                     
100%|██████████| 16270/16270 [26:39<00:00, 10.55it/s]

{'eval_loss': 1.2823090553283691, 'eval_runtime': 2.4658, 'eval_samples_per_second': 293.614, 'eval_steps_per_second': 73.404, 'epoch': 10.0}


100%|██████████| 16270/16270 [26:40<00:00, 10.16it/s]


{'train_runtime': 1600.713, 'train_samples_per_second': 40.651, 'train_steps_per_second': 10.164, 'train_loss': 0.9200849690170616, 'epoch': 10.0}


('./polarity_inversion_model/tokenizer_config.json',
 './polarity_inversion_model/special_tokens_map.json',
 './polarity_inversion_model/vocab.json',
 './polarity_inversion_model/merges.txt',
 './polarity_inversion_model/added_tokens.json',
 './polarity_inversion_model/tokenizer.json')

### Polarity Inversion Function
Defines a function to invert the polarity of a given review using the fine-tuned model.

In [ ]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check device availability
model.to(device)  # Move model to appropriate device

def invert_polarity(review):
    # Add the task-specific prefix
    input_text = "invert polarity: " + review.strip()  # Add prefix to the review
    
    # Encode the input text
    input_ids = tokenizer.encode(
        input_text, return_tensors='pt', truncation=True, max_length=128
    ).to(device)  # Convert input text to token IDs
    
    # Generate the output using the model
    outputs = model.generate(
        input_ids,
        max_length=128,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2,
    )  # Generate tokens for inverted polarity
    
    # Decode the generated tokens to get the inverted review
    inverted_review = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode token IDs to text
    return inverted_review  # Return the inverted review

### Testing Polarity Inversion
Tests the fine-tuned model on a list of sample reviews and displays the inverted polarity results.

In [ ]:
# Sample reviews to test
test_reviews = [
    "The product was bad",
    "This was the worst experience I've ever had.",
    "The service at this restaurant was fantastic and the food was delicious.",
    "I wouldn't recommend this to anyone. Complete waste of money.",
    "An amazing performance by the lead actor. Truly captivating!",
    "The software crashes frequently and is full of bugs.",
    "I am extremely satisfied with my purchase. Great value for the price.",
    "Terrible customer service. They were rude and unhelpful.",
    "Best vacation ever! The hotel staff was friendly and the amenities were top-notch.",
    "The book was bad.",
]

# Invert the polarity of each review and print the results
for review in test_reviews:
    inverted = invert_polarity(review)  # Get the inverted polarity
    print(f"Original Review: {review}")  # Print the original review
    print(f"Inverted Review: {inverted}")  # Print the inverted review

Original Review: The product was bad
Inverted Review: The product was fantastic!
--------------------------------------------------------------------------------
Original Review: This was the worst experience I've ever had.
Inverted Review: This was one of the best experiences I've ever had.
--------------------------------------------------------------------------------
Original Review: The service at this restaurant was fantastic and the food was delicious.
Inverted Review: The service at this restaurant was terrible and the food was awful.
--------------------------------------------------------------------------------
Original Review: I wouldn't recommend this to anyone. Complete waste of money.
Inverted Review: I highly recommend this to everyone. A fantastic investment of money!
--------------------------------------------------------------------------------
Original Review: An amazing performance by the lead actor. Truly captivating!
Inverted Review: A terrible performance by th

### Perplexity Computation
Calculates perplexity on test data to evaluate model performance.

In [ ]:
# Compute Perplexity on Test Data

import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    BartTokenizerFast,
    BartForConditionalGeneration,
)
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

# Load the fine-tuned model and tokenizer
tokenizer = BartTokenizerFast.from_pretrained('./polarity_inversion_model')
model = BartForConditionalGeneration.from_pretrained('./polarity_inversion_model')

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load the dataset
data = pd.read_csv('finaldataset.csv')

# Ensure all reviews are strings
data['original_review'] = data['original_review'].astype(str)
data['inverted_review'] = data['inverted_review'].astype(str)

# Add a task-specific prefix to the original reviews
data['input_text'] = 'invert polarity: ' + data['original_review']

# Convert the DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data[['input_text', 'inverted_review']])

# Split the dataset into training and test sets
tokenized_dataset = hf_dataset.train_test_split(test_size=0.1)
test_dataset = tokenized_dataset['test']

def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=128)
    outputs = tokenizer(examples['inverted_review'], padding='max_length', truncation=True, max_length=128)

    examples['input_ids'] = inputs.input_ids
    examples['attention_mask'] = inputs.attention_mask
    examples['labels'] = outputs.input_ids
    # Replace padding token id's of the labels by -100 so they are ignored in the loss computation
    examples['labels'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels]
        for labels in examples['labels']
    ]
    return examples

# Preprocess the test dataset
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=['input_text', 'inverted_review'])

columns = ['input_ids', 'attention_mask', 'labels']
test_dataset.set_format(type='torch', columns=columns)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=4)

# Set model to evaluation mode
model.eval()

total_loss = 0.0
total_tokens = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Computing Perplexity"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # This is the average loss over the batch

        # Number of tokens in labels that are not -100
        num_tokens = (labels != -100).sum().item()
        
        total_loss += loss.item() * num_tokens  # Multiply batch loss by number of tokens
        total_tokens += num_tokens

average_loss = total_loss / total_tokens
perplexity = np.exp(average_loss)

print(f"Total Loss: {total_loss:.4f}")
print(f"Total Tokens: {total_tokens}")
print(f"Average Loss per token: {average_loss:.4f}")
print(f"Perplexity: {perplexity:.4f}")


/home/toru/anaconda3/envs/polarity_env_pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Computing Perplexity: 100%|██████████| 181/181 [00:04<00:00, 43.62it/s]

Total Loss: 27789.5926
Total Tokens: 56738
Average Loss per token: 0.4898
Perplexity: 1.6320
